In [1]:
#%% library import
import numpy as np
import pandas as pd
import networkx as nx
import torch as tc
import torch
import pprint
import pickle
import time

from torch.autograd import Variable
from sklearn.utils import shuffle
from sklearn.preprocessing import Normalizer
from matplotlib import pyplot as plt
from matplotlib import image as img
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from functools import partial

In [2]:
#%% mini_dataset loading
with open("datasets/trainset_lstm+ecfp.txt", "rb") as fp:
    trainset = pickle.load(fp)

with open("datasets/validset_lstm+ecfp.txt", "rb") as fp:
    validset = pickle.load(fp)

Using device: cuda
GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [ ]:
#%% Make collate func.
def collate(samples):
    # The input `samples` is a list of pairs [(graph, label),(graph, label)].
    seqs, seq_lens, ecfps, labels = map(list, zip(*samples))
    return tc.LongTensor(seqs).cuda(), tc.LongTensor(seq_lens), tc.tensor(ecfps, dtype=tc.float).cuda(), tc.tensor(labels).cuda()

In [ ]:
#%% learning module 선언
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()    # method 상속받고 __init__()은 여기서 하겠다.
        
        self.emlayer = nn.Embedding(21, 10)
        self.lslayer = nn.LSTM(10, 64, num_layers=1, bidirectional=True, batch_first=True)
        
        self.eclayers = nn.Sequential(
                        nn.Linear(2048, 1024),
                        nn.BatchNorm1d(1024),
                        nn.ReLU(),
                        nn.Dropout(0.2),
                        nn.Linear(1024, 512),
                        nn.BatchNorm1d(512),
                        nn.ReLU(),
                        nn.Dropout(0.2),
                        nn.Linear(512, 256),
                        nn.BatchNorm1d(256),
                        nn.ReLU(),
                        nn.Dropout(0.2),
                        nn.Linear(256, 128),
                        )

        self.regress = nn.Linear(256, 1, F.elu)    # regression

    def forward(self, seq, seq_len, ecfp):
        sorted_seq_len, sorted_idx = seq_len.sort(0, descending=True)
        seq = seq[sorted_idx]
        
        ls_i = self.emlayer(seq)
        ls_i = pack_padded_sequence(ls_i, sorted_seq_len.tolist(), batch_first=True)
        ls_h = torch.zeros(2, 128, 64).cuda()     # (num_layers * num_directions, batch, hidden_size)
        ls_c = torch.zeros(2, 128, 64).cuda()
        
        ls_o, (ls_h, ls_c) = self.lslayer(ls_i, (ls_h, ls_c))
        ls_o, _ = pad_packed_sequence(ls_o, batch_first=True)

        # 순서 다시 바로잡아주기        
        _, sortedback_idx = sorted_idx.sort(0)
        ls_o = ls_o[sortedback_idx]
        
        # 각 sample의 last output vector 추출
        for_o = []
        for idx, o in enumerate(ls_o):
            for_o.append(o[seq_len[idx]-1, :64].view(1, 64))
        for_o = torch.cat(for_o, 0)
        
        back_o = ls_o[:, 0, 64:]
        
        concat_o = tc.cat((for_o, back_o), axis=1)   # batch, hidden*2
        
        ec_h = self.eclayers(ecfp)
        dim = 1
        for e in ec_h.size()[1:]:
            dim = dim * e
        ec_h = ec_h.view(-1, dim)
        
        cat = tc.cat((concat_o, ec_h), axis=1).cuda()
       
        return self.regress(cat).cuda()

In [ ]:
#%% Set hyperparameter
hp_d = {}

# FIXME: 학습 관련 하이퍼파라미터
hp_d['batch_size'] = 128
hp_d['num_epochs'] = 300

hp_d['init_learning_rate'] = 10 ** -3.70183
hp_d['eps'] = 10 ** -8.39981
hp_d['weight_decay'] = 10 ** -3.59967

In [ ]:
#%%
tr_data_loader = DataLoader(trainset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)
va_data_loader = DataLoader(validset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)

model = Regressor().to(torch.device('cuda:0'))
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
loss_func = nn.MSELoss(reduction='mean').cuda()
optimizer = optim.Adam(model.parameters(), lr=hp_d['init_learning_rate'], 
    weight_decay=hp_d['weight_decay'], eps=hp_d['eps'])

print('tr_var:', np.var(np.array([s[3] for s in trainset])))
print('va_var:', np.var(np.array([s[3] for s in validset])))
print('total params:', total_params)

tr_epoch_losses = []
va_epoch_losses = []

start = time.time()

for epoch in range(hp_d['num_epochs']):                          #!! epoch-loop
    # training session
    model.train()
    tr_epoch_loss = 0

    for iter, (seq, seq_len, ecfp, label) in enumerate(tr_data_loader):  #!! batch-loop
        prediction = model(seq, seq_len, ecfp).view(-1).cuda()
        loss = loss_func(prediction, label).cuda()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        tr_epoch_loss += loss.detach().item()
    
    tr_epoch_loss /= (iter + 1)
    print('Training epoch {}, loss {:.4f}'.format(epoch, tr_epoch_loss))
    tr_epoch_losses.append(tr_epoch_loss)

# ===========================================================================
    # validation session
    model.eval()
    va_epoch_loss = 0

    for iter, (seq, seq_len, ecfp, label) in enumerate(va_data_loader):  # batch-loop
        prediction = model(seq, seq_len, ecfp).view(-1).cuda()
        loss = loss_func(prediction, label).cuda()
        
        va_epoch_loss += loss.detach().item()
        
    va_epoch_loss /= (iter + 1)
    print('Validation epoch {}, loss {:.4f}'.format(epoch, va_epoch_loss))
    va_epoch_losses.append(va_epoch_loss)
    
end = time.time()
print('time elapsed:', end-start)


In [ ]:
np.save('lstm+ecfp_tr_losses', tr_epoch_losses)
np.save('lstm+ecfp_va_losses', va_epoch_losses)